In [1]:
import fastjet
import uproot
import awkward as ak
import hist
import numpy as np
import vector
vector.register_awkward()

In [2]:
def remove_muons(reco):
    '''Remove muons in a silly way:
    Remove any reco particle in the mass range 0.10566 +/- 0.03
    '''
    err = 0.03
    ll = reco.mass <= abs(0.10566-err)
    hl = reco.mass >= abs(0.10566+err)
    ll = ak.drop_none(ll)
    hl = ak.drop_none(hl)
    return reco[ll | hl]

In [3]:
redirector = 'root://eospublic.cern.ch//'
''' In case of problems, Download the root files from the following links:
https://prayag.web.cern.ch/share/FCC/eos/experiment/fcc/ee/generation/DelphesEvents/winter2023/IDEA/wzp6_ee_mumuH_Hbb_ecm240/events_159112833.root
'''

events_path = redirector+'eos/experiment/fcc/ee/generation/DelphesEvents/winter2023/IDEA/wzp6_ee_mumuH_Hbb_ecm240/events_159112833.root:events'

def _remove_not_interpretable(branch, emit_warning=True):
    if isinstance(
        branch.interpretation, uproot.interpretation.identify.uproot.AsGrouped
    ):
        for name, interpretation in branch.interpretation.subbranches.items():
            if isinstance(
                interpretation, uproot.interpretation.identify.UnknownInterpretation
            ):
                if emit_warning:
                    warnings.warn(
                        f"Skipping {branch.name} as it is not interpretable by Uproot"
                    )
                return False
    if isinstance(
        branch.interpretation, uproot.interpretation.identify.UnknownInterpretation
    ):
        if emit_warning:
            warnings.warn(
                f"Skipping {branch.name} as it is not interpretable by Uproot"
            )
        return False

    try:
        _ = branch.interpretation.awkward_form(None)
    except uproot.interpretation.objects.CannotBeAwkward:
        if emit_warning:
            warnings.warn(
                f"Skipping {branch.name} as it is it cannot be represented as an Awkward array"
            )
        return False
    else:
        return True

events = uproot.open(
    events_path,
    open_files=False,
    filter_name =  lambda x : "PARAMETERS" not in x,
    filter_branch = _remove_not_interpretable
)

In [4]:
Reco = ak.zip(
    {
        'E':events['ReconstructedParticles/ReconstructedParticles.energy'].arrays()['ReconstructedParticles.energy'],
        'px':events['ReconstructedParticles/ReconstructedParticles.momentum.x'].arrays()['ReconstructedParticles.momentum.x'],
        'py':events['ReconstructedParticles/ReconstructedParticles.momentum.y'].arrays()['ReconstructedParticles.momentum.y'],
        'pz':events['ReconstructedParticles/ReconstructedParticles.momentum.z'].arrays()['ReconstructedParticles.momentum.z'],
    },
    with_name='Momentum4D'
)
              

pseudo_jets = remove_muons(Reco)

#Ensure there are at least 2 reco particles before clustering
pseudo_jets = pseudo_jets[ak.num(pseudo_jets, axis=1) > 1]

In [5]:
# Custom setup for jupyter environments like jupyterhub

import os
import sys
import subprocess

# Equivalent of determining the location of the setup script
LOCAL_DIR = os.path.dirname(os.path.abspath(""))
os.environ["LOCAL_DIR"] = LOCAL_DIR

# Add scripts directory to PYTHONPATH
scripts_path = os.path.join(LOCAL_DIR, "scripts")
if scripts_path not in sys.path:
    sys.path.insert(0, scripts_path)

# Set PYTHONPATH environment variable
if "PYTHONPATH" in os.environ:
    os.environ["PYTHONPATH"] = f"{scripts_path}:{os.environ['PYTHONPATH']}"
else:
    os.environ["PYTHONPATH"] = scripts_path


cmd = "python3 -c 'import fastjet; print(fastjet.__path__[0])'"
fastjet_path = subprocess.check_output(cmd, shell=True).decode().strip()
fastjet_lib_path = os.path.join(fastjet_path, "_fastjet_core/lib")

# Update LD_LIBRARY_PATH
if "LD_LIBRARY_PATH" in os.environ:
    os.environ["LD_LIBRARY_PATH"] = f"{os.environ['LD_LIBRARY_PATH']}:{fastjet_lib_path}"
else:
    os.environ["LD_LIBRARY_PATH"] = fastjet_lib_path

print(f"LOCAL_DIR set to: {os.environ['LOCAL_DIR']}")
print(f"PYTHONPATH now includes: {os.environ.get('PYTHONPATH', '')}")
print(f"LD_LIBRARY_PATH now includes: {os.environ.get('LD_LIBRARY_PATH', '')}")

LOCAL_DIR set to: /home/cms-jovyan/coffea-fcc-analyses
PYTHONPATH now includes: /home/cms-jovyan/coffea-fcc-analyses/scripts
LD_LIBRARY_PATH now includes: /opt/conda/lib/:/usr/local/lib/python3.12/site-packages/fastjet/_fastjet_core/lib


## Ignoring the stuff above, we now have pseudojets available to play around

In [6]:
from plugins.fastjet import Recombiner

In [7]:
recomb = Recombiner.E0_Scheme.instance()
recomb

<Swig Object of type 'fastjet::JetDefinition::Recombiner *' at 0x7f1295246e50>

In [8]:
def get_jets(pseudojets):
    
    jetdef = fastjet.JetDefinition0Param(fastjet.ee_kt_algorithm)
    jetdef.set_recombiner(recomb)

    cluster = fastjet.ClusterSequence(pseudo_jets, jetdef)
    
    return cluster.exclusive_jets(2), cluster.exclusive_jets_constituents(2)

In [9]:
jets, jet_constituents = get_jets(pseudo_jets)

dijets = ak.sum(jets, axis=1)

h = hist.Hist.new.Reg(100,0,200).Double().fill(dijets.mass)
h

#--------------------------------------------------------------------------
#                         FastJet release 3.4.2
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


Hist(Regular(100, 0, 200, label='Axis 0'), storage=Double()) # Sum: 99987.0 (100000.0 with flow)